# Chapter4

In [1]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [2]:
%cd /content/drive/MyDrive/bioinf/podaci/

/content/drive/MyDrive/bioinf/podaci


In [13]:
from collections import Counter, defaultdict
import re

In [4]:
with open('/content/drive/MyDrive/bioinf/zadatak/genetic_code.txt', 'r') as f:
  gen_code = [line.strip().split(':') for line in f.readlines()]

def get_genetic_code(reverse=False):
  if reverse:
    G = {}
    for pair in gen_code:
      if pair[1] not in G:
        G[pair[1]] = [pair[0]]
      else:
        G[pair[1]].append(pair[0])
    return G
  else:
    G = {}
    for pair in gen_code:
      G[pair[0]] = pair[1]
    return G

In [8]:
with open('/content/drive/MyDrive/bioinf/zadatak/mass.txt', 'r') as f:
  lines = [line.strip().split(':') for line in f.readlines()]
def get_mass():
  G = {}
  for pair in lines:
    G[pair[0]] = int(pair[1])
  return G

#    

## [BA4A](https://rosalind.info/problems/ba4a/) - Translate an RNA String into an Amino Acid String

In [11]:
def translation(rna_pattern):
  G = get_genetic_code() # kopija rijecnika rna - amino kiselina
  res = [] # za spremanje odgovora
  for i in range(0, len(rna_pattern), 3): # iteriranje po uzorku
    # 3 znaka u uzorku odgovaraju jednoj amino kiselini
    letter = G[rna_pattern[i : i + 3]] # pronadi amino kiselinu od substringa
    if letter == "*": # ako je aminokiselina jednaka * - stop
      break # prekini petlju
    res.append(letter) # dodaj amino kiselinu u niz
  res = "".join(res) # formatiraj odgovor
  return res # vrati odgovor

translation('AUGGCCAUGGCGCCCAGAACUGAGAUCAAUAGUACCCGUAUUAACGGGUGA')

'MAMAPRTEINSTRING'

In [12]:
with open("rosalind_ba4a.txt", "r") as f:
  text = f.readline().strip()
translation(text)

'MRVRMGGELFIISGASRVSLWANRSGDGSGPIPMIDKQQLKLSRLRRPCVAHSLVKLHIQLFGVRVARVPWCRNGVKLVPREHHRSVILLKSTDFRQAGVSRLCSNHLAFIGTEPTALTLKRKAILWGALRLLRLRFYRLRCVRAKVEALLRGGQLQQVSAIQSRAGRFDCKHTARIRYPRRGDPKQLLGRRCRCTAMSDKLLEGYPSNIRRGERSGARGAHGWSAMNVSKAVAPMETGKTVVDICRVACLFIISRKISASAGDGACMEQNFCSLVKYYYHRGWDMLFTLIAGHALRIPHRPAVPVQRLPDQIVQIRHGQEKYPLRPLQVKHLGTVKIALCIMTPAQSQGRFHFAQRTRLWGSGLIREIPLRHKITHYRKGIERPRGDDWCCLGRYRVRSRRTTLVICRSLLPVKVSGFFAGIAPTAFLNCTIGSPQRVDNIYDRILEHRPSFRKCSGTRLPGPGLSQTTSPSSPSPSQVSSSGGIFRQWSQVARLRIWSIEAREPSPYRASQGLWKESSSADLWRAITNAWLYSQCTDREVGVSALRLRMWISDCPSWPSYDAPPAMRCTIGRGHYATRVGMLQPFEAFASVDLQATEDNGVAILKINWRHWPYVNAISQVGMPKDFFHDRPRSVSRSLDADVTTTVLDGSIICRVSAVTTLSQGLFARYERRVPFIARSMLWEIFRAWITSGISKSESESEPQGRPASDVGAVYHILETRGAIRDARSSRSTVRCRTQSLRQAIRPDMTMPHLRRHVRPGSARASVRNNYALRCEPVIQLVFNTESWFSAREQARTHKAKILGLSSQATDMSNQVDTLDHQKWGDVQHDRAQEQRSGSSLDRDPLISFSICFPRTASRFEFSVLKCRMTTNVFRRLEHLYVYVTFQNTKGRDIPWPHYQNIYVGLAQWSHACDAIERHSPEIYMFVTASKFPSLKSEACTLLYPSWKRCVAALLSHTLSPARTLTLQCGCWIGRIYVLRILCSLWLLTVSPGSATKA

## [BA4B](https://rosalind.info/problems/ba4b/) - Find Substrings of a Genome Encoding a Given Amino Acid String

In [15]:
def reverseComplement(text):
  complement = {"A": "T", "T": "A", "C": "G", "G": "C"} # komplementi u dna
  # pronadi komplement za svaki element u obrnutom stringu
  return "".join([complement[x] for x in text[::-1]])

In [14]:
def dnaToRna(text):
  return re.sub("T", "U", text) # pretvori DNA u RNA zamjenom T u U

In [16]:
def getAllCombinationsWorker(peptide, codons_one_to_three,
                                current_status, all_combinations): # dobivanje kombinacija za peptif
  # kada prodemo cijeli peptid, dodajemo u all_combinations
  if len(peptide) == 0: # ako je duljina peptida jednaka 0
    # u sve kombinacije nadodaj trenutni status
    all_combinations.append(current_status)
    return # vrati
  # za svaki kodon in u kodonima od prvog elementa peptida
  for codon in codons_one_to_three[peptide[0]]:
    # rekurzivno dohvati slovo iz peptida
    # za svako slovo trazi troslove (rna) i spaja
    getAllCombinationsWorker(peptide[1:], codons_one_to_three,
                                current_status + codon, all_combinations)
# npr T -> nade sve moguce i stavi ih u niz
# zatim npt prelazi na slovo P i nade sve komb i spoji

In [17]:
def getAllCombinations(peptide):
  # kreiraj rijecnik
  codons_one_to_three = get_genetic_code(reverse=True)
  current_status = "" # tenutni status
  all_combinations = [] # sve kombinacije za spremanje
  # dohvati sve kombinacije
  getAllCombinationsWorker(peptide, codons_one_to_three,
                              current_status, all_combinations)
  return all_combinations # vrati sve kombinacije

In [18]:
def findAll(pattern, text):
  i = text.find(pattern) # pronadi index uzorka u stringu
  while i != -1: # dok je i u stringu (razlicit od -1)
    yield i # vrati i -> yeald -> pamti podatke i vraca odjednom
    i = text.find(pattern, i + 1) # nastavljamo trazit po tekstu od ite pozicije

#vraca mjesta u tekstu na kojima pocinje pattern

In [21]:
def getAllAminoAcidCandidates(dna, peptide):
  # broj znakova u obrnutom peptidu
  n = len(peptide) * 3 # postavi n na broj znakova u peptidu * 3
  rcdna = reverseComplement(dna) # pronadi obrnuti komplement od dobivenog dna
  rna = dnaToRna(dna) # napravi odgovarajuci rna od dna
  rcrna = dnaToRna(rcdna) # napravi rna od komplementa dna
  # dohvati sve kombinacije za peptide - kandidate
  candidates = getAllCombinations(peptide)
  valid = []
  for cand in candidates: # za svakog kandidata u kandidatima
    # trazimo indexe di se nalazi kandidat
    for index in findAll(cand, rna): # za svaki index u rna
      valid.append(dna[index : index + n]) # dodaj dna od indexa do index + n
      continue # nastavi
    for index in findAll(cand, rcrna): # za svaki index u komplement rna
      # dodaj obrnuti komplement od indexa do indexa +n
      valid.append(reverseComplement(rcdna[index : index + n]))
  return '\n'.join(valid)

ans = getAllAminoAcidCandidates('ATGGCCATGGCCCCCAGAACTGAGATCAATAGTACCCGTATTAACGGGTGA', 'MA')
print(ans)

ATGGCC
ATGGCC
GGCCAT


In [22]:
with open("rosalind_ba4b.txt", "r") as f:
  dna = f.readline().strip()
  peptide =f.readline().strip()
getAllAminoAcidCandidates(dna, peptide)

'CTGTACATTTCGCAGTTGCCATCC\nGGATGGCAATTACGGAACGTACAA\nCTGTACATTTCGAAGCTGCCATCC\nCTGGACATTCCTTAGTTGCCAGCC\nGGCTGGCAACTACGCAACGTCCAA\nCTGGACGTTTCGCAGTTGCCAGCC\nTTGCACATTTCTAAGTTGCCAGCC\nGGCTGGCAATTGAGAAATGTTCAG\nGGCTGGCAATTGCGAAACGTACAG\nGGCTGGCAGCTACGGAACGTGCAA\nCTGTACATTCCGTAGCTGCCACCC\nCTGCACATTACGTAGCTGCCACCC\nTTGAACGTTGCGAAGCTGCCACCC\nGGGTGGCAGTTGCGTAATGTTCAG\nGGTTGGCAACTACGAAACGTACAA\nTTGAACGTTGCGCAATTGCCAACC\nGGTTGGCAGCTCCGAAACGTACAG\nTTGGACATTGCGCAGCTGCCAACC\nTTGAACGTTCCTTAACTGCCAACC\nGGTTGGCAGTTACGAAATGTACAA'

## [BA4C](https://rosalind.info/problems/ba4c/) - Generate the Theoretical Spectrum of a Cyclic Peptide

- **Peptid:** LEQN
- **Ciklički peptid:** LEQNLE
- **Subpeptidi:**  L, LE, LEQ, E, EQ, EQN, Q, QN, QNL, N, NL, NLE
- **Spekter:** 0, 484, 113 ,242 ,370 ,129 ,257 ,371 ,128 ,242 ,355 ,114 ,227 ,356
- **Sortirani spektar:** 0, 113, 114, 128, 129, 227, 242, 242, 257, 355, 356, 370, 371, 484

In [31]:
def spectrum(peptide):
  n = len(peptide) # n je dljina peptida
  mass = get_mass() # radimo kopiju masa aminokiselina
  # produlji peptid kopijom peptid do predzadnjeg znaka
  extended_peptide = peptide + peptide[:-2]  # jer je ciklicni peptid
  spectrum = [0, sum([mass[x] for x in peptide])] # spektar masa
  # dvi petlje kako bi dobili sve kombinacije peptida
  for l in range(n): # za svako slovo u peptidu
    for k in range(1, n): # za svako slovo u peptidu od 1 do n
      subpeptide = extended_peptide[l : l + k] # uzmi subpeptid od l do l+k
      # u spektar dodaj sumu mase za subpeptid
      spectrum.append(sum([mass[x] for x in subpeptide]))
  spectrum.sort() # sortirani spektar
  return ' '.join(str(x) for x in spectrum)

peptide ='LEQN'
spectrum(peptide)

'0 113 114 128 129 227 242 242 257 355 356 370 371 484'

In [26]:
with open("rosalind_ba4c.txt", "r") as f:
  peptide = f.readline().strip()
spectrum(peptide)

'0 71 87 87 101 101 103 103 114 129 129 129 137 172 188 190 208 215 215 216 216 232 232 232 266 286 291 302 309 316 319 319 335 337 345 353 361 387 403 405 420 422 423 424 438 448 448 464 474 482 506 523 524 525 534 551 551 552 553 577 577 585 611 635 637 638 639 652 653 654 656 680 706 714 714 738 739 740 740 757 766 767 768 785 809 817 827 843 843 853 867 868 869 871 886 888 904 930 938 946 954 956 972 972 975 982 989 1000 1005 1025 1059 1059 1059 1075 1075 1076 1076 1083 1101 1103 1119 1154 1162 1162 1162 1177 1188 1188 1190 1190 1204 1204 1220 1291'

##[BA4D](https://rosalind.info/problems/ba4d/) - Compute the Number of Peptides of Given Total Mass

In [32]:
def getAllCombinationsWorker(mass, aa_masses, counter):
  if mass in counter: # ako je masa u brojacu
    return counter[mass] # vrati brojac od mase
  if mass < 0: # ako je masa manja od 0
    return 0 # vrati 0
  if mass == 0: # ako je masa jednaka 0
    return 1 # vrati 1
  counter[mass] = 0 # postavi brojac od mase na 0
  for aa_mass in aa_masses: # za svaku amino masu u masama aminokiselina
    # nadodaj u brojac za masu sve kombinacije tako da masu
    # smanjujemo s masom aminokiseline
    counter[mass] += getAllCombinationsWorker(mass - aa_mass,
                                                 aa_masses, counter)
  return counter[mass] #vrati brojac od mase

In [35]:
def getAllCombinations(mass):
  # dohvati sve mase aminokiselina
  aa_masses = set(get_mass().values())
  counter = {} # brojac
  # dohvati brojac od mase
  res = getAllCombinationsWorker(mass, aa_masses, counter)
  # print({k: v for k,v in counter.items() if v!=0}) #ispisi dobivene brojace
  return res

getAllCombinations(1024)

14712706211

In [37]:
with open("rosalind_ba4d.txt", "r") as f:
  mass = f.readline().strip()
getAllCombinations(int(mass))

13269154757756

##[BA4E](https://rosalind.info/problems/ba4e/) - Find a Cyclic Peptide with Theoretical Spectrum Matching an Ideal Spectrum

In [40]:
#BA4C
def cyclospectrum(peptide): # definicija ciklospektra
  n = len(peptide) # peptid duljine n
  mass = get_mass() # dohvatimo mase
  # prolazi i do  -2 -> abcd -> abcdab -> ciklicnost
  extended_peptide = peptide + peptide[:-1] # prosirimo peptid
  spectrum = [0, sum(peptide)] # u spektar dodamo nulu i cijelu sumu
  for l in range(n): # iteriramo po listi
    for k in range(1, n): # dujine peptida
      subpeptide = extended_peptide[l : l + k] # trazimo sve podpeptide
      spectrum.append(sum(subpeptide)) # dodajemo sumu u spektar
  return sorted(spectrum) # vracamo sortirani spektar

In [41]:
def linearspectrum(peptide): # zelimo da je konzistentan s ciklickim
  n = len(peptide) # duzina peptida
  mass = get_mass() # dohvati mase aminokiselina
  # postavi spektar na 0 // bez ukupne mase - razlika od prijasnjeg def
  spectrum = [0]
  for l in range(n): # za svaki n u dužini spektra
    for k in range(1, n - l + 1): # za svaki k od 1 do n-l+1
      # ne smijemo prijec preko peptida !!
      subpeptide = peptide[l : l + k] # napravi podpeptid
      spectrum.append(sum(subpeptide)) # u spektar dodaj sume subpeptida
  return sorted(spectrum) # vrati sortirani spektar

In [42]:
# je li linearni spektar konzistentan s ciklickim
def belongs_spectrum(candidate_spectrum, main_spectrum):
  # spektar kandidata iz linearning - racunamo broj pojavljivanja
  # rijecnik rijesenje
  counter_c_spectrum = Counter(candidate_spectrum)
  # spektar ciklicnog spektra
  counter_m_spectrum = Counter(main_spectrum)
  # za svaki kljuc vrijednost(broj pojavljivanja) iz kandidata
  for key, count in counter_c_spectrum.items():
    # ako nema kljuca (peptida) u ciklicnom spektru
    if key not in counter_m_spectrum:
      return False # vrati false
    # ako je broj pojavljivanja veci u linearnom od ciklicog za peptid
    if count > counter_m_spectrum[key]:
      return False # vrati false
  return True # inace vrati true

In [43]:
# ispituje masu i spektar kandidata
def check_cyclo(spectrum, peptide):
  parent_mass = max(spectrum) # masa spektra
  # ako je masa peptida jednaka sumi (masi spektra)
  # ako je ciklospektar peptida jednak spektru
  if sum(peptide) == parent_mass and cyclospectrum(peptide) == spectrum:
    return True #vrati istina
  return False # vrati laz

In [44]:
def check_linear(spectrum, peptide): #provjeri linearnost
  parent_mass = max(spectrum) # dohvati maksimum spektra
  # ako je zbroj svih peptida manji ili jednak masi maksimuma spektra
  # i ako linearni spektar peptida je konzistentan sa spektrom
  if sum(peptide) <= parent_mass and belongs_spectrum(
      linearspectrum(peptide), spectrum):
    return True #vrati true
  return False # vrati false

In [49]:
def branch_and_bound(spectrum):
  all_combinations = [] # niz svih kombinacija
  # dohvati sve mase aminokiselina - skup
  aa_masses = set(get_mass().values())
  # dohvati najvecu masu iz spektra(masa cijelog spektra)
  parent_mass = max(spectrum)
  peptides = [[]] #matrica peptida
  while True: # dok je istina
    # kandidati -> dodajemo pocetnim peptidima sve moguce mase
    # za svaki peptid iz liste peptida
    # na pocetku lista prazna pa sve se mase dodaju
    candidates = [peptide + [aa_mass] for aa_mass
                  in aa_masses for peptide in peptides]
    # provjeravamo zadovoljavaju konzistentnost, ako jesu dodajemo ih
    # inace izbacujemo
    selected = [peptide for peptide in candidates
                if check_linear(spectrum, peptide)]
    # u sve kombinacije dodajemo peptide iz spektra koji su konzistentnzi
    # provjeravamo masu i konzistentnost spektra kandidata
    all_combinations.extend([peptide for peptide in
                             selected if check_cyclo(spectrum, peptide)])
    # ubacujemo sve peptide iz izabranih kojima suma nije jednaka sumi peptida
    # koja nije jednaka -> ako je jednaka onda s njima nemamo sta radit
    peptides = [peptide for peptide in
                selected if sum(peptide) != parent_mass]
    if len(peptides) == 0: # kad je duljina peptida 0 - sve smo ih izbacili
      break # prekinut cemo petlju
    res = []
  for comb in all_combinations:
    res.append('-'.join(str(x) for x in comb))
  return ' '.join(sorted(res))

spectrum = '0 113 128 186 241 299 314 427'
spectrum = [int(x) for x in spectrum.split(' ')]
branch_and_bound(spectrum)

'113-128-186 113-186-128 128-113-186 128-186-113 186-113-128 186-128-113'

In [50]:
with open("rosalind_ba4e.txt", "r") as f:
  spectrum = f.readline().split()
spectrum = [int(x) for x in spectrum]
branch_and_bound(spectrum)

'103-103-114-87-156-128-115-147 103-103-147-115-128-156-87-114 103-114-87-156-128-115-147-103 103-147-115-128-156-87-114-103 114-103-103-147-115-128-156-87 114-87-156-128-115-147-103-103 115-128-156-87-114-103-103-147 115-147-103-103-114-87-156-128 128-115-147-103-103-114-87-156 128-156-87-114-103-103-147-115 147-103-103-114-87-156-128-115 147-115-128-156-87-114-103-103 156-128-115-147-103-103-114-87 156-87-114-103-103-147-115-128 87-114-103-103-147-115-128-156 87-156-128-115-147-103-103-114'

## [BA4F](https://rosalind.info/problems/ba4f/) - Compute the Score of a Cyclic Peptide Against a Spectrum

In [51]:
def findSpectrum(peptide):
  n = len(peptide) # n je duljina peptida
  mass = get_mass() # radimo kopiju masa aminokiselina
  # produlji peptid kopijom peptid do predzadnjeg znaka
  extended_peptide = peptide + peptide[:-2]  # jer je ciklicni peptid
  spectrum = [] # spektar masa
  spectrum.append(0) # dodaj pocetnu nulu
  # zbroji mase iz rijecnika za svako slovo u peptidu
  # i dodaj u spektar
  spectrum.append(sum([mass[x] for x in peptide]))
  # dvi petlje kako bi dobili sve kombinacije peptida
  for l in range(n): # za svako slovo u peptidu
    for k in range(1, n): # za svako slovo u peptidu od 1 do n
      subpeptide = extended_peptide[l : l + k] # uzmi subpeptid od l do l+k
      # u spektar dodaj sumu mase za subpeptid
      spectrum.append(sum([mass[x] for x in subpeptide]))
  return sorted(spectrum) # vrati sortirani spektar

In [52]:
# slican score-u iz BA4G
def score(peptide, spectrum_):
  pep_spec = findSpectrum(peptide) # teoretski spektar ciklopeptida
  result = 0 # postavi rezultat na 0
  # jedinstvene mase su skup (brojevi se ne ponavljaju)
  # teoretskog spektra i dobivenog spektra
  unique_masses = set(pep_spec + spectrum_)
  for mass in unique_masses: # za svaku masu u jedinstvenim masama
    # u rezultat pribroji -> prebroji pojavljivanja u teoretskom spektru
    # ciklopeptida i dobivenom spektru te izmedu ta dva odaberi minimalni
    # broj pojavljivanja
    result += min(pep_spec.count(mass), spectrum_.count(mass))
  return result # vrati rezultat

score('NQEL',[0, 99, 113, 114, 128, 227, 257, 299, 355, 356, 370, 371, 484])

11

In [53]:
with open("rosalind_ba4f.txt", "r") as f:
  peptide = f.readline().strip()
  spect = f.readline().split()
spectrum_ = [int(x) for x in spect]
score(peptide, spectrum_)

698

## [BA4G](https://rosalind.info/problems/ba4g/) - Implement LeaderboardCyclopeptideSequencing

In [59]:
# BA4E
def cyclospectrum(peptide): # definicija ciklospektra
  n = len(peptide) # peptid duljine n
  mass = get_mass() # dohvatimo mase
  # prolazi i do  -2 -> abcd -> abcdab -> ciklicnost
  extended_peptide = peptide + peptide[:-1] # prosirimo peptid
  spectrum = [0, sum(peptide)] # u spektar dodamo nulu i cijelu sumu
  for l in range(n): # iteriramo po listi
    for k in range(1, n): # dujine peptida
      subpeptide = extended_peptide[l : l + k] # trazimo sve podpeptide
      spectrum.append(sum(subpeptide)) # dodajemo sumu u spektar
  return sorted(spectrum) # vracamo sortirani spektar

In [60]:
def expand(peptides):
  # dohvati sve mase aminokiselina - skup
  aa_masses = set(get_mass().values())
  # novi peptidi -> dodajemo pocetnim peptidima sve
  # moguce mase za svaki peptid iz liste peptida
  # na pocetku lista prazna pa sve se mase dodaju
  new_peptides = [peptide + [aa_mass] for aa_mass
                  in aa_masses for peptide in peptides]
  return new_peptides # vrati nove peptidea

In [61]:
def Score(peptide, spectrum):
    pep_spec = cyclospectrum(peptide) # mase u ciklospektru
    result = 0 # postavi rezultat na 0
    # jedinstvene mase su skup (brojevi se ne ponavljaju)
    # teoretskog spektra i dobivenog spektra
    unique_masses = set(pep_spec + spectrum)
    for mass in unique_masses: # za svaku masu u jedinstvenim masama
      # u rezultat pribroji -> prebroji pojavljivanja u teoretskom spektru
      # ciklopeptida i dobivenom spektru te izmedu ta dva odaberi minimalni
      # broj pojavljivanja
      result += min(pep_spec.count(mass), spectrum.count(mass))
    return result # vrati rezultat

In [62]:
def Trim(leaderboard, spectrum, N):
  if len(leaderboard) <= N: # ako je duzina ljestvice manja ili  jednaka N
    return leaderboard # vrati ljestvicu
  scores = {} # rijecnik postignuca
  # za svaku poziciju i i masu peptida u ljestvici
  for i, peptide in enumerate(leaderboard):
    # postavi index kao kljuc, a vrijednost Score peptida u spektru
    scores[i] = Score(peptide, spectrum)
  # napravi sortianu listu vrijednosti iz scores silazno
  sorted_scores = sorted(scores.values(), reverse=True)
  # postavi kao prag N-1-vi element iz sortiranih postignuca
  threshold = sorted_scores[N - 1]
  # vrati mase peptida iz ljestvice koji imaju veci score od praga
  # pronadi index, score iz originalne ljestvice unutar scores i
  # usporedi s granciom
  return [leaderboard[idx] for idx, score
          in scores.items() if score >= threshold]

In [64]:
def leaderboard_cyclopeptide_sequencing(spectrum, N):
  leaderboard = [[]] # ljestvica je matrica
  leader_peptide = [] # vodeci peptid
  while leaderboard:
    leaderboard = expand(leaderboard) # prosiri ljestvicu
    for peptide in leaderboard: # za svaki peptid u ljestvici
      # zbroji mase peptida i ako su one jednake
      # zadnjoj masi spektra(ukupna masa spektra)
      if sum(peptide) == spectrum[-1]:
        # ako je score peptida napram spektru vece od score-a vodeceg peptida
        # napram spektru
        if Score(peptide, spectrum) > Score(leader_peptide, spectrum):
          # postavi da je vodeci peptid onaj peptid koji ima veci score
          leader_peptide = peptide
      # inace ako je suma masa veca od ukupne sume spektra
      elif sum(peptide) > spectrum[-1]:
        # ljestvica je jednaka svim peptidima u ljestvici koji su
        # razliciti od trenutnog peptida koji ima vecu masu
        leaderboard = [pep for pep in leaderboard if pep != peptide]
    # vrati N peptida
    leaderboard = Trim(leaderboard, spectrum, N)
  return '-'.join(str(x) for x in leader_peptide) # vrati vodeci peptid

ans = leaderboard_cyclopeptide_sequencing([0, 71, 113, 129, 147,
                                     200, 218, 260, 313, 331,
                                     347, 389, 460], 10)
print(ans)

113-147-71-129


In [65]:
with open("rosalind_ba4g.txt", "r") as f:
  N = int(f.readline().strip())
  spect = f.readline().split()
spectrum_ = [int(x) for x in spect]
res = leaderboard_cyclopeptide_sequencing(spectrum_, N)
res

'137-113-113-147-163-163-101-113-115-101-99-129-128'

## [BA4H](https://rosalind.info/problems/ba4h/) - Generate the Convolution of a Spectrum

In [67]:
def convolution(spectrum):
  spectrum.sort() # sortiraj spektar
  conv = [] # konvolucija
  # sve kombinacije elemenata spektra i njihove razlike
  for i in range(len(spectrum) - 1): # za svaku poziciju u spektru
    for j in range(i, len(spectrum)): # za svaku poziciju od i do kraja spektra
      # ako je razlika od i-tog elementa spektra i j-tog elementa
      # masenog spektra razlicita od nula
      if spectrum[j] - spectrum[i] != 0:
        # u listu konv. dodaj njihovu razliku
        conv.append(spectrum[j] - spectrum[i])
  freq_dict = {} # rijecnik frekvencija
  for mass in set(conv): # za svaku masu u konv koja nema duplih elemenata
    # dodaj kao kljuc masu, a kao vrijednost prebroji mase iz konv.
    freq_dict[mass] = conv.count(mass)
  # stvori popis kljuceva iz rjecnika freq_dict, sortiranih silaznim
  # redoslijedom prema njihovim odgovarajucim vrijednostima.
  sorted_mass_list = [k for k, _ in sorted(freq_dict.items(),
                                           key=lambda item: item[1],
                                           reverse=True)]
  conv = []
  for mass in sorted_mass_list: # za svaku masu u listi sortiranih masa
    # na conv nadodaj masu onoliko puta kolika joj je frekvenvija
    conv += [mass] * freq_dict[mass]
  return ' '.join([str(x) for x in conv]) # vrati mase

ans = convolution([0, 137, 186, 323])
print(ans)

137 137 186 186 323 49


In [68]:
with open("rosalind_ba4h.txt", "r") as f:
  spect = f.readline().split()
spectrum_ = [int(x) for x in spect]
res = convolution(spectrum_)
res

'87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 87 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113

## [BA4I](https://rosalind.info/problems/ba4i/) - Implement ConvolutionCyclopeptideSequencing

In [69]:
# BA4E
def cyclospectrum(peptide): # definicija ciklospektra
  n = len(peptide) # peptid duljine n
  mass = get_mass() # dohvatimo mase
  # prolazi i do  -2 -> abcd -> abcdab -> ciklicnost
  extended_peptide = peptide + peptide[:-1] # prosirimo peptid
  spectrum = [0, sum(peptide)] # u spektar dodamo nulu i cijelu sumu
  for l in range(n): # iteriramo po listi
    for k in range(1, n): # dujine peptida
      subpeptide = extended_peptide[l : l + k] # trazimo sve podpeptide
      spectrum.append(sum(subpeptide)) # dodajemo sumu u spektar
  return sorted(spectrum) # vracamo sortirani spektar

In [70]:
# BA4H
def convolution(spectrum):
  spectrum.sort() # sortiraj spektar
  conv = [] # konvolucija
  # sve kombinacije elemenata spektra i njihove razlike
  for i in range(len(spectrum) - 1): # za svaku poziciju u spektru
    for j in range(i, len(spectrum)): # za svaku poziciju od i do kraja spektra
      # ako je razlika od i-tog elementa spektra i j-tog elementa
      # masenog spektra razlicita od nula
      if spectrum[j] - spectrum[i] != 0:
        # u listu konv. dodaj njihovu razliku
        conv.append(spectrum[j] - spectrum[i])
  freq_dict = {} # rijecnik frekvencija
  for mass in set(conv): # za svaku masu u konv koja nema duplih elemenata
    # dodaj kao kljuc masu, a kao vrijednost prebroji mase iz konv.
    freq_dict[mass] = conv.count(mass)
  # stvori popis kljuceva iz rjecnika freq_dict, sortiranih silaznim
  # redoslijedom prema njihovim odgovarajucim vrijednostima.
  sorted_mass_list = [k for k, _ in sorted(freq_dict.items(),
                                           key=lambda item: item[1],
                                           reverse=True)]
  conv = []
  for mass in sorted_mass_list: # za svaku masu u listi sortiranih masa
    # na conv nadodaj masu onoliko puta kolika joj je frekvenvija
    conv += [mass] * freq_dict[mass]
  return conv # vrati mase

In [71]:
# iz BA4E -> candidates
def Expand(peptides, masses):
  # novi peptidi -> dodajemo pocetnim peptidima sve
  # moguce mase za svaki peptid iz liste peptida
  # na pocetku lista prazna pa sve se mase dodaju
  new_peptides = [peptide + [mass] for mass
                  in masses for peptide in peptides]
  return new_peptides # vrati nove peptidea

In [72]:
# BA4G
def Score(peptide, spectrum):
    pep_spec = cyclospectrum(peptide) # mase u ciklospektru
    result = 0 # postavi rezultat na 0
    # jedinstvene mase su skup (brojevi se ne ponavljaju)
    # teoretskog spektra i dobivenog spektra
    unique_masses = set(pep_spec + spectrum)
    for mass in unique_masses: # za svaku masu u jedinstvenim masama
      # u rezultat pribroji -> prebroji pojavljivanja u teoretskom spektru
      # ciklopeptida i dobivenom spektru te izmedu ta dva odaberi minimalni
      # broj pojavljivanja
      result += min(pep_spec.count(mass), spectrum.count(mass))
    return result # vrati rezultat

In [73]:
# BA4G
def Trim(leaderboard, spectrum, N):
  if len(leaderboard) <= N: # ako je duzina ljestvice manja ili  jednaka N
    return leaderboard # vrati ljestvicu
  scores = {} # rijecnik postignuca
  # za svaku poziciju i i masu peptida u ljestvici
  for i, peptide in enumerate(leaderboard):
    # postavi index kao kljuc, a vrijednost Score peptida u spektru
    scores[i] = Score(peptide, spectrum)
  # napravi sortianu listu vrijednosti iz scores silazno
  sorted_scores = sorted(scores.values(), reverse=True)
  # postavi kao prag N-1-vi element iz sortiranih postignuca
  threshold = sorted_scores[N - 1]
  # vrati mase peptida iz ljestvice koji imaju veci score od praga
  # pronadi index, score iz originalne ljestvice unutar scores i
  # usporedi s granciom
  return [leaderboard[idx] for idx, score
          in scores.items() if score >= threshold]

In [74]:
def find_masses(spectrum, M):
  conv = convolution(spectrum) # napravi konvoluciju spektra
  # samo ostavi one elemente iz konvolucije koji su veci ili jednaki 57
  # i koji su manji ili jednaki 200
  conv = [x for x in conv if 57 <= x <= 200]
  freq_dict = {}
  for mass in set(conv): # za svaku jedinstvenz masu iz konvolucije
    # prebroji frekvencije pojavljivanja mase u konvoluciji
    freq_dict[mass] = conv.count(mass)
  # stvori popis kljuceva iz rjecnika freq_dict, sortiranih silaznim
  # redoslijedom prema njihovim odgovarajucim vrijednostima
  sorted_elems = sorted(freq_dict.items(), key=lambda kv: kv[1], reverse=True)
  # uzima popis torki sorted_elems kao ulaz i vraća popis masa gdje je
  # frekvencija mase veca ili jednaka frekvenciji M-tog najcesceg
  # elementa u sorted_elems
  masses = [mass for mass, freq in sorted_elems if freq >= sorted_elems[M-1][1]]
  masses.sort() # sortiraj mase
  return masses # vrati mase

In [76]:
# slicno iz BA4G
def convolution_cyclopeptide_sequencing(spectrum, M, N):
  masses = find_masses(spectrum, M) # pronadi mase
  leaderboard = [[]] # ljestvica je matrica
  leader_peptide = [] # vodeci peptid
  while leaderboard:
    leaderboard = Expand(leaderboard, masses) # prosiri ljestvicu
    for peptide in leaderboard: # za svaki peptid u ljestvici
      # zbroji mase peptida i ako su one jednake
      # zadnjoj masi spektra(ukupna masa spektra)
      if sum(peptide) == spectrum[-1]:
        # ako je score peptida napram spektru vece od score-a vodeceg peptida
        # napram spektru
        if Score(peptide, spectrum) > Score(leader_peptide, spectrum):
          # postavi da je vodeci peptid onaj peptid koji ima veci score
          leader_peptide = peptide
      # inace ako je suma masa veca od ukupne sume spektra
      elif sum(peptide) > spectrum[-1]:
        # ljestvica je jednaka svim peptidima u ljestvici koji su
        # razliciti od trenutnog peptida koji ima vecu masu
        leaderboard = [pep for pep in leaderboard if pep != peptide]
    # vrati N peptida
    leaderboard = Trim(leaderboard, spectrum, N)
  return "-".join(str(el) for el in leader_peptide) # vrati vodeci peptid

ans = convolution_cyclopeptide_sequencing([57, 57, 71, 99, 129, 137, 170,
                                           186, 194, 208, 228, 265, 285,
                                           299, 307, 323, 356, 364, 394,
                                           422, 493], 20, 60)
print(ans)

137-71-99-57-72-57


In [77]:
with open("rosalind_ba4i.txt", "r") as f:
  M = int(f.readline().strip())
  N = int(f.readline().strip())
  spect = f.readline().split()
spectrum_ = [int(x) for x in spect]
res = convolution_cyclopeptide_sequencing(spectrum_, M, N)
res

'57-113-113-103-129-147-113-101-71-184-57-71-115'

## [BA4J](https://rosalind.info/problems/ba4j/) - Generate the Theoretical Spectrum of a Linear Peptide

In [79]:
def linearSpectrum(peptide, external = False):
  masses = get_mass() # dohvati mase
  spectrum = [0] # postavi nulu u spektar
  n = len(peptide) # dohvati duljinu peptida
  for i in range(n):
    for j in range(1, n-i+1):
      subpeptide = peptide[i: i+j] # uzmi subpeptide linearnog peptida
      # zbroji mase subpeptida i dodaj ih u spektar
      spectrum.append(sum([masses[x] for x in subpeptide]))
  spectrum.sort() # sortiraj spektar
  if external:
    return spectrum
  return ' '.join(str(x) for x in spectrum)

peptide = 'NQEL'
linearSpectrum(peptide)

'0 113 114 128 129 242 242 257 370 371 484'

In [81]:
with open("rosalind_ba4j.txt", "r") as f:
  peptide = f.readline().strip()
res = linearSpectrum(peptide)
res

'0 57 57 71 71 71 97 97 97 97 97 99 101 101 101 113 115 115 128 128 128 128 128 128 128 128 129 129 129 131 137 137 137 137 147 154 156 163 163 163 163 168 186 194 194 194 198 199 199 199 218 225 225 225 226 229 229 230 230 234 234 241 243 243 246 251 255 256 258 264 264 265 265 269 274 276 278 287 291 291 293 296 296 296 300 322 323 326 326 344 345 346 347 348 349 353 354 355 362 362 365 371 371 374 378 380 383 392 392 392 393 393 393 393 393 397 402 405 406 406 424 433 443 449 450 454 456 463 464 468 473 475 476 477 477 477 481 482 483 489 490 493 494 499 499 502 502 507 507 515 520 521 530 534 534 540 546 551 555 569 570 572 573 577 578 578 578 587 591 592 597 599 601 604 605 608 611 611 612 618 619 622 627 627 630 644 652 667 669 670 671 671 679 679 683 688 689 692 697 698 698 701 701 701 706 706 709 726 728 732 737 739 740 740 740 741 741 745 747 755 758 764 768 771 789 794 795 798 798 807 808 811 816 826 827 829 829 834 836 842 842 842 846 846 848 857 860 860 868 869 869 869 873 

## [BA4K](https://rosalind.info/problems/ba4k/) - Compute the Score of a Linear Peptide

In [82]:
def linear_score(peptide, spectrum):
  # napravi linearni spektar peptida
  pep_spec = linearSpectrum(peptide, external = True)
  result = 0 # postavi rezultat na 0
  # pronadi sve jedinstvene mase u linearnom spektru i dobivenom spektru
  unique_masses = set(pep_spec + spectrum)
  for mass in unique_masses: # za svaku masu u jedinstvenim masama
    # dodaj (zbroji) minimum pojavljivanja odredene mase izmedu pojavljivanja
    # u linearnom spektru peptida i dobivenom spektru
    result += min(pep_spec.count(mass), spectrum.count(mass))
  return result # vrati rezultat

ans = linear_score('NQEL', [0, 99, 113, 114, 128, 227, 257,
                            299, 355, 356, 370, 371, 484])
print(ans)

8


In [83]:
with open("rosalind_ba4k.txt", "r") as f:
  peptide = f.readline().strip()
  spect = f.readline().split()
spectrum_ = [int(x) for x in spect]
res = linear_score(peptide, spectrum_)
res

340

## [BA4L](https://rosalind.info/problems/ba4l/) - Trim a Peptide Leaderboard

In [84]:
#BA4J
def LinearSpectrum(Peptide):
  mass_table = get_mass() # dohvati mase aminokiselina
  PrefixMass = [0] # postavi prefiks na niz koji sadrzi 0 -> uvijek krece s 0
  for i in range(len(Peptide)): # za svaki element u duljini peptida
    # na prefiks nadodaj masu peptida koji se nalazi na i-tom indeksu
    temp = PrefixMass[i] + mass_table[Peptide[i]]
    # nadodaj u listu temp
    PrefixMass.append(temp)
  linearSpectrum = [0]  # postavi linearni spektar na niz koji sadrzi 0
  for i in range(len(Peptide)): # za svaki element u peptidu
    # za svaki element u peptidu od i+1 pozicije
    for j in range(i + 1, len(Peptide) + 1):
      # nadodaj u linearni spektar razliku masa od i-tog i j-tog
      # elementa prefiks masa
      linearSpectrum.append(PrefixMass[j] - PrefixMass[i])
  linearSpectrum.sort() # sortiraj uzlazno
  return linearSpectrum # vrati linearni spektar

In [85]:
# BA4K
def linear_score(peptide, spectrum):
  pep_spec = LinearSpectrum(peptide) # napravi linearni spektar peptida
  result = 0 # postavi rezultat na 0
  # pronadi sve jedinstvene mase u linearnom spektru i dobivenom spektru
  unique_masses = set(pep_spec + spectrum)
  for mass in unique_masses: # za svaku masu u jedinstvenim masama
    # dodaj (zbroji) minimum pojavljivanja odredene mase izmedu pojavljivanja
    # u linearnom spektru peptida i dobivenom spektru
    result += min(pep_spec.count(mass), spectrum.count(mass))
  return result # vrati rezultat

In [87]:
def Trim(leaderboard, spectrum, N):
  if len(leaderboard) <= N: # ako je duzina ljestvice manja ili  jednaka N
    return leaderboard # vrati ljestvicu
  scores = {} # rijecnik postignuca
  # za svaku poziciju i i masu peptida u ljestvici
  for i, peptide in enumerate(leaderboard):
    # postavi index kao kljuc, a vrijednost linearnog Score-a peptida u spektru
    scores[i] = linear_score(peptide, spectrum)
  # napravi sortianu listu vrijednosti iz scores silazno
  sorted_scores = sorted(scores.values(), reverse=True)
  # postavi kao prag N-1-vi element iz sortiranih postignuca
  threshold = sorted_scores[N - 1]
  # vrati mase peptida iz ljestvice koji imaju veci score od praga
  # pronadi index, score iz originalne ljestvice unutar scores i
  # usporedi s granciom
  return " ".join([leaderboard[idx] for idx, score
          in scores.items() if score >= threshold])

ans = Trim(['LAST', 'ALST', 'TLLT', 'TQAS'],
 [0, 71, 87, 101, 113, 158, 184, 188, 259, 271, 372], 2)
print(ans)

LAST ALST


In [88]:
with open("rosalind_ba4l.txt", "r") as f:
  leaderboard = f.readline().split()
  spect = f.readline().split()
  N = int(f.readline().strip())
spectrum_ = [int(x) for x in spect]
res = Trim(leaderboard, spectrum_, N)
res

'AYWQTVEDWNRQPKDQVEDNTQFPYGNNQAPVTNYPTEVHDIRFTSQADN PPILRCDSKDTQHALMSIEDLGGEMQRNHDYDHIRHNPRVEAQTYYESQT RRTCCDITHILYKGFMCLQAKCTCTEKDLFPHIYYGHGGGELAVTPSTLL FTTPHLGPCEPVGTFVYFFLDFFVFVLADLGDPAAAYVRGLIERGSWTEW CPWWWWKLKFTISKMPTPTPKKATQKNQAESGPLWTESCPKYHMGMGYTN'

## [BA4M](https://rosalind.info/problems/ba4m/) - Solve the Turnpike Problem

**Turnpike problem:** D = 2 2 3 3 4 5 6 7 8 10 ->dobijemo listu međusobnih udaljenosti između točaka

- prvo označimo najudaljenije točke na pravcu (uvijek postavljamo 0 kao početni)

  0----**10**
  
  - D = 2 2 3 3 4 5 6 7 8 **10** -> D = 2 2 3 3 4 5 6 7 8

- uzmemo sljedeći najveći i pokušamo ga umetnuni s lijeva (od 0)

  0---**8**-10 -> udaljenosti: od 8 do 10 -> **2**, od 8 do 0 -> **8**
  
  - D = **2** 2 3 3 4 5 6 7 **8** ->  D = 2 3 3 4 5 6 7

- uzimamo sljedeći najveći i pokušamo ga umetnuti s lijeva (od 0)

  0--**7**-8-10 -> udaljenosti: od 7 do 10 -> **3**, od 7 do 0 -> **7**, od 7 do 8 -> **1** -> u D ne postoji 1 -> pokušat ćemo umetnuti 7 s desna

  0-**3**--8-10 -> udaljenosti : od 3 do 10 -> **7**, od 3 do 0 -> **3**, od 3 do 8 -> **5**

  - D = 2 **3** 3 4 **5** 6 **7** -> D = 2 3 4 6

- uzimamo sljedeći najveći i pokušamo ga umetnuti s lijeva (od 0)

  0-3-**6**-8-10 -> udaljenosti: od 6 do 10 -> **4**, od 6 do 0 -> **6**, od 6 do 8 -> **2**, od 6 do 3 -> **3**

  - D = **2** **3** **4** **6** -> D = /

**Rješenje:** 0 3 6 8

In [112]:
def delta(point, line):
  line = list(line) # skup udaljenosti
  dist = []
  for line_point in line: # za svaku tocku
    # izracunaj udaljenost od odabrane tocke i tocke na liniji
    dist.append(abs(line_point-point))
  return dist # vrati udaljenosti

In [113]:
def is_subset(temp, L):
  for dist in temp: # za svaku udaljenost iz temp
    # provjeri nalazi li se u pocetnim udaljenostima
    if temp.count(dist) > L.count(dist):
      return False
  return True

In [122]:
def difference(temp, L):
  diffset = [] # azurirani skup udaljenosti
  c_temp = Counter(temp) # prebroji sve udaljenosti iz temp
  c_L = Counter(L) # prebroji sve udaljenosti u pocetnom skupu
  for dist, count in c_L.items(): # za svaku udaljenost u pocetnom
    diff = count - c_temp.get(dist, 0) # izracunaj razliku pojavljivanja
    if diff > 0: # ako je razlika veca od nula
      # nadodaj udaljenost onoliko puta kolika je razlika
      for i in range(diff):
        diffset.append(dist)
  diffset.sort() # sortiaj
  return diffset

In [123]:
def PlaceDot(L, line, width):
  if len(L) == 0: # ako nema udaljensoti
    return line # vrati liniju
  ## pokusaj umetnuti tocku s lijeva
  point = L[-1] # uzmi najvecu vrijednost iz udaljenosti
  temp = delta(point, line) # izracunaj udaljenosti od tocke i tocaka na pravcu
  # ako se udaljenosti od tocke nalaze u pocetnim udaljenostima
  if is_subset(temp, L):
    line.add(point) # dodaj tocku
    # azuriranje stanja s udaljenostima nakon dodavanja nove tocke
    L_left = difference(temp, L)
    L_res = PlaceDot(L_left, line, width) # rekurzivno smjestanje s novim stanjem
    if L_res:# ako skup tocaka nije prazan
      return L_res # vrati skup kao rjesenje
    line.remove(point) # inace ukloni tocku
  ## pokusaj umetnuti tocku s desna
  temp = delta(width - point, line)
  if is_subset(temp, L):
    line.add(width - point)
    L_right = difference(temp, L)
    R_res = PlaceDot(L_right, line, width)
    if R_res:
      return R_res
    line.remove(width - point)
  return {}

In [124]:
def turnpike(L):
  L = [int(x) for x in L.split(' ') if int(x) > 0]
  L.sort()
  width = L.pop(-1) # najudaljenija tocka od 0
  line = {0, width} # postavi pocetni raspored
  res = PlaceDot(L, line, width) # smjesti tocke
  return ' '.join(str(x) for x in res)

L = '-10 -8 -7 -6 -5 -4 -3 -3 -2 -2 0 0 0 0 0 2 2 3 3 4 5 6 7 8 10'
turnpike(L)

'0 3 6 8 10'

In [127]:
with open("rosalind_ba4m.txt", "r") as f:
  L = f.readline()

turnpike(L)

'0 1024 514 1283 515 1288 264 781 270 18 1042 1044 788 22 1303 534 25 282 536 1309 1055 34 1061 294 807 1064 551 43 304 817 51 564 1081 571 829 64 320 576 323 68 580 839 1100 1102 79 336 595 84 85 343 855 91 861 1119 607 354 100 613 359 105 1131 1132 109 878 1135 375 887 631 122 892 1149 893 130 387 648 397 654 1167 912 145 403 405 661 408 921 155 1180 924 162 1187 419 677 1190 935 680 426 428 1197 173 1201 182 953 698 957 1214 446 1218 197 965 969 457 717 1230 975 1232 211 471 220 733 991 735 994 1251 230 1255 490 236 749 494 748 1265 241 1010 244 755 1015 248 1017 763 510 1023'